# Importing libraries

In [ ]:
pip install "tensorflow_hub>=0.6.0"

In [ ]:
!pip install --upgrade tensorflow_hub

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import sigmoid_kernel
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf

# loading & viewing the dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data sets/tmdb_movies_data.csv')
df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,101199955.5,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,183999919.0,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,174799923.1,1.385749e+09


In [ ]:
df.shape

(10866, 21)

# Simple dropping for nulls & duplicates

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  9836 non-null   object 
 15  release_date       

In [ ]:
df.duplicated().sum()

1

In [ ]:
df = df.dropna(subset=['overview', 'genres'])
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)
df.shape

(10839, 21)

# Fitting the ELMO vectorizer based on overview of the movie

In [ ]:
 tf.compat.v1.disable_v2_behavior()

 elmo = hub.load("https://tfhub.dev/google/elmo/3")

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
def elmo_vectors(x, batch_size=32):
    embeddings_list = []
    for i in range(0, len(x), batch_size):
        batch = x[i: i + batch_size].tolist()
        embeddings = elmo.signatures['default'](tf.constant(batch))["elmo"]
        with tf.compat.v1.Session() as sess:
            sess.run(tf.compat.v1.global_variables_initializer())
            sess.run(tf.compat.v1.tables_initializer())
            embeddings_list.append(sess.run(tf.reduce_mean(embeddings, 1)))

    return np.concatenate(embeddings_list, axis=0)

In [ ]:
# Fitting the ELMO vectorizer on the 'overview' text
elmo_matrix = elmo_vectors(df['overview'])
elmo_matrix.shape

(10839, 1024)

# Using sigmoid to find similarity

In [ ]:
sig = sigmoid_kernel(elmo_matrix, elmo_matrix)
sig[0]

array([0.7634166 , 0.7642572 , 0.7627377 , ..., 0.7629924 , 0.76338816,
       0.7630745 ], dtype=float32)

# Using cosine similarity

In [ ]:
cosine_sim = cosine_similarity(elmo_matrix, elmo_matrix)
cosine_sim[0]

array([0.9999994 , 0.4528572 , 0.6471482 , ..., 0.43929625, 0.65694207,
       0.7156614 ], dtype=float32)

In [ ]:
df.index[df['original_title']=='The Dark Knight Rises']

Index([4350], dtype='int64')

In [ ]:
sorted(list(enumerate(cosine_sim[0])), reverse=True, key=lambda x: x[1])[1:6]

[(5980, 0.82393765),
 (1105, 0.8184635),
 (5668, 0.8155961),
 (5693, 0.813597),
 (3596, 0.81230825)]

# Giving recommendations based on sigmoid

In [ ]:
def give_rec_sig(movie_title, sig=sig):
    # Get the index corresponding to original_title
    idx = df[df['original_title']== movie_title].index[0]

    # Sort the movies & get the scores of the 10 most similar movies
    sig_scores = sorted(list(enumerate(sig[idx])), reverse=True, key=lambda x: x[1])[1:11]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df['original_title'].iloc[movie_indices]

In [ ]:
print(give_rec_sig('Star Wars: The Force Awakens'))

6176    Star Wars: Episode III - Revenge of the Sith
3904    Star Wars: Episode II - Attack of the Clones
3375                                 Kung Fu Panda 2
5630                                   Super Buddies
2165               Family Guy Presents: It's a Trap!
5412                                     Pacific Rim
3748               è³½å¾·å…‹Â·å·´èŠ (ä¸Š)  å¤ªé™½æ——
7871                             Conan the Destroyer
4702                       Mass Effect: Paragon Lost
8743                                The 10th Kingdom
Name: original_title, dtype: object


# Giving recommendations based on cosine similarity

In [ ]:
def give_rec_cos(movie_title, cos=cosine_sim):
    # Get the index corresponding to original_title
    idx = df[df['original_title']== movie_title].index[0]

    # Sort the movies & get the scores of the 10 most similar movies
    cos_scores = sorted(list(enumerate(cos[idx])), reverse=True, key=lambda x: x[1])[1:11]

    # Movie indices
    movie_indices = [i[0] for i in cos_scores]

    # Top 10 most similar movies
    return df['original_title'].iloc[movie_indices]

In [ ]:
print(give_rec_cos('Star Wars: The Force Awakens'))

10497                          The Transformers: The Movie
3596                        Green Lantern: Emerald Knights
7297                                          Flash Gordon
7902     The Adventures of Buckaroo Banzai Across the 8...
5438                        Percy Jackson: Sea of Monsters
10045                   American Ninja 4: The Annihilation
7850                                             Stridulum
816                                    Justice League: War
10741                        The Star Wars Holiday Special
8493                                 Hellraiser: Bloodline
Name: original_title, dtype: object
